In [ ]:
!pip install ollama

In [ ]:
# 1. Install BOTH dependencies (zstd for unzipping, pciutils for your A100)
!sudo apt-get update && sudo apt-get install -y zstd pciutils

# 2. Install Ollama (Now it will succeed and see your GPU)
!curl -fsSL https://ollama.com/install.sh | sh

# 3. Start the Server
import subprocess
import time

print("Starting Ollama server...")
# Run as a background process
process = subprocess.Popen("ollama serve", shell=True)

# Give it 10 seconds to initialize the GPU
time.sleep(10)

# 4. Download the Model
print("Downloading Llama 3.1 8B...")
!ollama pull llama3.1:8b

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://cli.github.com/packages stable InRelease [3,917 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [61.5 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,205 kB]
Get:13 https://cloud.

In [ ]:
!ollama pull llava:7b
!ollama pull mistral

In [ ]:
import json
import glob
import os
from tqdm import tqdm
import ollama

# 1. Configuration
MODEL_NAME = "llama3.1:8b"
MODEL_NAME = "llava:7b"
MODEL_NAME = "mistral"
SEED_VALUE = 42

print(f"Targeting model: {MODEL_NAME} | Seed: {SEED_VALUE}")

input_pattern = "*.jsonl"
files = glob.glob(input_pattern)

print(f"Found {len(files)} files: {files}")

for input_file in files:

    if "_results.jsonl" in input_file:
        continue

    base_name = os.path.splitext(input_file)[0]
    output_file = f"{base_name}_results.jsonl"

    with open(input_file, 'r', encoding='utf-8') as f:
        data_list = [json.loads(line) for line in f]

    results = []

    print(f"\nStarting: {input_file}")
    for item in tqdm(data_list, desc="Generating Answers"):

        user_prompt = item.get("prompt", "")

        messages = [
            {"role": "user", "content": user_prompt},
        ]

        try:
            response = ollama.chat(
                model=MODEL_NAME,
                messages=messages,
                options={
                    'temperature': 0.7,
                    'num_predict': 256,
                    'seed': SEED_VALUE, # Passed here
                }
            )

            generated_text = response['message']['content']

        except Exception as e:
            print(f"Error: {e}")
            generated_text = "[ERROR]"

        item["model_response"] = generated_text
        results.append(item)

    with open(output_file, 'w', encoding='utf-8') as f:
        for entry in results:
            f.write(json.dumps(entry) + '\n')

    print(f"Finished {input_file} -> Saved to {output_file}")

print("\nAll tasks completed.")

Targeting model: mistral | Seed: 42
Found 10 files: ['safety.jsonl', 'tricky.jsonl', 'factual_accuracy.jsonl', 'insufficient_information_results.jsonl', 'neutral.jsonl', 'insufficient_information.jsonl', 'tricky_results.jsonl', 'safety_results.jsonl', 'factual_accuracy_results.jsonl', 'neutral_results.jsonl']

Starting: safety.jsonl


Generating Answers: 100%|██████████| 40/40 [01:12<00:00,  1.80s/it]


Finished safety.jsonl -> Saved to safety_results.jsonl

Starting: tricky.jsonl


Generating Answers: 100%|██████████| 40/40 [00:36<00:00,  1.09it/s]


Finished tricky.jsonl -> Saved to tricky_results.jsonl

Starting: factual_accuracy.jsonl


...

Finished factual_accuracy.jsonl -> Saved to factual_accuracy_results.jsonl

Starting: neutral.jsonl


Generating Answers: 100%|██████████| 40/40 [00:27<00:00,  1.47it/s]


Finished neutral.jsonl -> Saved to neutral_results.jsonl

Starting: insufficient_information.jsonl


Generating Answers: 100%|██████████| 40/40 [00:43<00:00,  1.10s/it]

Finished insufficient_information.jsonl -> Saved to insufficient_information_results.jsonl

All tasks completed.
